### Importing data gathered from twitter

In [1]:
import os
import sys
import glob

data_directory = os.path.join(os.path.dirname(os.getcwd()),'data/twitter')

files = glob.glob(os.path.join(data_directory,'*.csv'))
files

['/Users/sami/Projects/vote2017/data/twitter/fillon.csv',
 '/Users/sami/Projects/vote2017/data/twitter/hamon.csv',
 '/Users/sami/Projects/vote2017/data/twitter/lepen.csv',
 '/Users/sami/Projects/vote2017/data/twitter/macron.csv',
 '/Users/sami/Projects/vote2017/data/twitter/melenchon.csv',
 '/Users/sami/Projects/vote2017/data/twitter/valls.csv']

In [2]:
import pandas as pd
import csv
from gensim import corpora
from gensim.models.word2vec import LineSentence
sys.path.append(os.path.dirname(os.getcwd()))
from utils.preprocessing import preprocess_sentence

corpus = {}

for file in files:
    print('reading ',file)
    df = pd.read_csv(file, sep=';', quoting=csv.QUOTE_ALL)
    text = df.text.values
    tweets = []
    for t in text:
        tweets.append(preprocess_sentence(t))
    name = file.split('/')[-1]
    corpus[name] = tweets

No handlers could be found for logger "gensim.models.word2vec"


SyntaxError: Non-ASCII character '\xc3' in file /Users/sami/Projects/vote2017/utils/preprocessing.py on line 16, but no encoding declared; see http://python.org/dev/peps/pep-0263/ for details (preprocessing.py, line 15)

In [ ]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

## Building Dictionaries

Removing rare words (appearing in less than 20 tweets or in more than 50% of the tweets).

In [ ]:
from gensim.corpora import Dictionary

dictionaries = {}
for k,v in corpus.items():
    d = Dictionary(v)
    d.filter_extremes(no_below=10, no_above=0.4)
    dictionaries[k] = d
                                 

## Building LDA Model

In [ ]:
from gensim.models import LdaModel

name = 'fillon.csv'
#Bag of words representation of the tweets
c = [dictionaries[name].doc2bow(doc) for doc in corpus[name]]

print('Number of unique tokens: %d' % len(dictionaries[name]))
print('Number of documents: %d' % len(corpus[name]))

# Set training parameters.
num_topics = 10
chunksize = 2000
passes = 20
iterations = 400
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionaries[name][0]  # This is only to "load" the dictionary.
id2word = dictionaries[name].id2token

%time model = LdaModel(corpus=c, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)


In [ ]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim.prepare(model, corpus[name], dictionaries[name])

## Word2vec model